In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("./cmd/addr.txt",header=None,sep='\t',dtype=str,names=['type','code','name'])

In [8]:
province = df[df['type']=='province']

In [21]:
def process(row):
    name = row['name']
    if name.endswith('市'):
        return [name, name.rstrip('市')]
    elif name.endswith('省'):
        return [name, name.rstrip('省')]
    elif name == '内蒙古自治区':
        return [name, '内蒙古省', '内蒙古', '内蒙']
    elif name == '广西壮族自治区':
        return [name, '广西自治区', '广西省', '广西']
    elif name == '西藏自治区':
        return [name, '西藏省', '西藏']
    elif name == '新疆维吾尔自治区':
        return [name,'新疆维吾尔族自治区', '新疆自治区', '新疆省', '新疆']
    return [name]

In [23]:
province['ext'] = province.apply(process, axis=1)

/Users/hwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
province

,type,code,name,ext
0,province,110000000000,北京市,"[北京市, 北京]"
1,province,120000000000,天津市,"[天津市, 天津]"
2,province,130000000000,河北省,"[河北省, 河北]"
3,province,140000000000,山西省,"[山西省, 山西]"
4,province,150000000000,内蒙古自治区,"[内蒙古自治区, 内蒙古省, 内蒙古, 内蒙]"
5,province,210000000000,辽宁省,"[辽宁省, 辽宁]"
6,province,220000000000,吉林省,"[吉林省, 吉林]"
7,province,230000000000,黑龙江省,"[黑龙江省, 黑龙江]"
8,province,310000000000,上海市,"[上海市, 上海]"
9,province,320000000000,江苏省,"[江苏省, 江苏]"
